<a href="https://colab.research.google.com/github/aebongbing/ESAA/blob/main/240429_ESAA_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import glob, os

# 다음은 저자의 컴퓨터에서 압축 파일을 풀어놓은 디렉터리이니, 각자 디렉터리를 다시 설정한다.
path =r'/content/drive/MyDrive/opinosis+opinion+frasl+review/OpinosisDataset1.0/topics'
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list =[]
opinion_text =[]

# 개별 파일의 파일명은 filename_list로 취합
# 개별 파일의 파일 내용은 DataFrame로딩 후 다시 string으로 변환해 opinion_text list로 취합
for file_ in all_files:
  # 개별 파일을 읽어서 DataFrame으로 생성
  df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
  # 절대 경로로 주어진 파일명을 가공. 리눅스에서 수행할 때는 다음\를 /로 변경
  # 맨 마지막 .data 확장자도 제거
  filename_ = file_.split('/')[-1]
  filename = filename_.split('.')[0]

  # 파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가
  filename_list.append(filename)
  opinion_text.append(df.to_string())

# 파일명 list와 파일 내용 list 객체를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,battery-life_amazon_kindle,...
2,buttons_amazon_kindle,...
3,bathroom_bestwestern_hotel_sfo,...
4,battery-life_ipod_nano_8gb,...


In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

nltk.download('punkt')
nltk.download('wordnet')

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# 문서 TF-IDF 형태로 피처 벡터화
# LemNormalize() 이용
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect=TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',ngram_range=(1,2),min_df=0.05, max_df=0.85)

In [ ]:
#opinion_text 칼럼 값으로 픽처벡터화 수행
feature_vect=tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [ ]:
#kmeans수행 후 군집의 label값과 중심별로 할당된 데이터셋의 좌표 구하기
from sklearn.cluster import KMeans

#5개의 집합으록 군집화 수행
km_cluster=KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label=km_cluster.labels_
cluster_centers=km_cluster.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
#각 데이터별 할당된 군집의 레이블을 파일명과 파일내용 가지고 있는 df에 cluster_label 칼럼 추가해 저장
document_df['cluster_label']=cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,3
1,battery-life_amazon_kindle,...,2
2,buttons_amazon_kindle,...,1
3,bathroom_bestwestern_hotel_sfo,...,0
4,battery-life_ipod_nano_8gb,...,2


In [ ]:
#호텔 리뷰 군집화
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
3,bathroom_bestwestern_hotel_sfo,...,0
14,food_holiday_inn_london,...,0
9,food_swissotel_chicago,...,0
17,free_bestwestern_hotel_sfo,...,0
20,location_bestwestern_hotel_sfo,...,0
23,location_holiday_inn_london,...,0
21,parking_bestwestern_hotel_sfo,...,0
30,price_holiday_inn_london,...,0
25,room_holiday_inn_london,...,0
33,rooms_bestwestern_hotel_sfo,...,0


In [ ]:
#전자기기 리뷰 군집화
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
2,buttons_amazon_kindle,...,1
8,eyesight-issues_amazon_kindle,...,1
10,features_windows7,...,1
13,fonts_amazon_kindle,...,1
19,keyboard_netbook_1005ha,...,1
22,navigation_amazon_kindle,...,1
38,screen_garmin_nuvi_255W_gps,...,1
34,screen_ipod_nano_8gb,...,1
37,screen_netbook_1005ha,...,1
44,size_asus_netbook_1005ha,...,1


In [ ]:
#차량용네비게이션 군집화
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
1,battery-life_amazon_kindle,...,2
4,battery-life_ipod_nano_8gb,...,2
6,battery-life_netbook_1005ha,...,2
29,performance_netbook_1005ha,...,2
46,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,2
47,video_ipod_nano_8gb,...,2


In [ ]:
#호텔 리뷰 군집화
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,3
11,directions_garmin_nuvi_255W_gps,...,3
7,display_garmin_nuvi_255W_gps,...,3
35,satellite_garmin_nuvi_255W_gps,...,3
40,speed_garmin_nuvi_255W_gps,...,3
49,updates_garmin_nuvi_255W_gps,...,3
50,voice_garmin_nuvi_255W_gps,...,3


In [ ]:
#자동차 리뷰 군집화
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
5,comfort_honda_accord_2008,...,4
12,comfort_toyota_camry_2007,...,4
15,gas_mileage_toyota_camry_2007,...,4
18,interior_honda_accord_2008,...,4
16,interior_toyota_camry_2007,...,4
24,mileage_honda_accord_2008,...,4
27,performance_honda_accord_2008,...,4
28,price_amazon_kindle,...,4
26,quality_toyota_camry_2007,...,4
31,seats_honda_accord_2008,...,4


In [ ]:
#군집개수 약간 많이 설정돼있어 세분화되어 군집화된 경향있음 -> 중심개수 5에서 3으로 낮춰서 3개 그룹으로 군집화한뒤 확인
from sklearn.cluster import KMeans

#3개의 집합으로 군집화
km_cluster=KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label=km_cluster.labels_
cluster_centers=km_cluster.cluster_centers_

#소속 군집을 cluster_label칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label']=cluster_label
document_df.sort_values(by='cluster_label')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
47,video_ipod_nano_8gb,...,0
46,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,0
44,size_asus_netbook_1005ha,...,0
41,speed_windows7,...,0
40,speed_garmin_nuvi_255W_gps,...,0
38,screen_garmin_nuvi_255W_gps,...,0
37,screen_netbook_1005ha,...,0
35,satellite_garmin_nuvi_255W_gps,...,0
34,screen_ipod_nano_8gb,...,0


In [ ]:
#전자기기 리뷰 군집화
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
1,battery-life_amazon_kindle,...,0
4,battery-life_ipod_nano_8gb,...,0
6,battery-life_netbook_1005ha,...,0
2,buttons_amazon_kindle,...,0
11,directions_garmin_nuvi_255W_gps,...,0
7,display_garmin_nuvi_255W_gps,...,0
8,eyesight-issues_amazon_kindle,...,0
10,features_windows7,...,0
13,fonts_amazon_kindle,...,0


In [ ]:
#호텔 리뷰 군집화
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
5,comfort_honda_accord_2008,...,1
12,comfort_toyota_camry_2007,...,1
15,gas_mileage_toyota_camry_2007,...,1
18,interior_honda_accord_2008,...,1
16,interior_toyota_camry_2007,...,1
24,mileage_honda_accord_2008,...,1
27,performance_honda_accord_2008,...,1
26,quality_toyota_camry_2007,...,1
31,seats_honda_accord_2008,...,1
42,transmission_toyota_camry_2007,...,1


In [ ]:
cluster_centers=km_cluster.cluster_centers_
print('cluster_centers shape: ',cluster_centers.shape)
print(cluster_centers)

cluster_centers shape:  (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


In [19]:
#군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함
def get_cluster_details(cluster_model, cluster_data, cluster_nums, feature_names, top_n_features=10):

    cluster_details = {}

    #cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값 반환
    #군집중심점별 할당된 word 피처들의 거리값이 큰순으로 값을 구하기 위함
    centroid_feature_ordered_ind = cluster_model.cluster_centers_ .argsort()[:, ::-1] # 행별(군집별)로 역순 정렬

    #개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
    for cluster_num in range(cluster_nums):
        # 군집별 정보를 담을 데이터 초기화
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster']=cluster_num

        #cluster_centers_.argsort()[:,::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함
        top_feature_indexes=centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features=[feature_names[ind] for ind in top_feature_indexes]

        #top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
        top_feature_values=cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

        #cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심 위치 상댓값, 해당 파일명 입력
        cluster_details[cluster_num]['top_features']=top_features
        cluster_details[cluster_num]['top_features_value']=top_feature_values
        filenames=cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
        filenames=filenames.values.tolist()

        cluster_details[cluster_num]['filenames']=filenames

    return cluster_details

In [20]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('###### Cluster {0}'.format(cluster_num))
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명:',cluster_detail['filenames'][:7])
    print('===============================================')

In [21]:
feature_names = tfidf_vect.get_feature_names_out()
cluster_details=get_cluster_details(cluster_model=km_cluster, cluster_data=document_df, feature_names=feature_names, cluster_nums=3, top_n_features=10)
print_cluster_details(cluster_details)

###### Cluster 0
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명: ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'buttons_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'display_garmin_nuvi_255W_gps', 'eyesight-issues_amazon_kindle']
###### Cluster 1
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명: ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_toyota_camry_2007', 'interior_honda_accord_2008', 'mileage_honda_accord_2008', 'quality_toyota_camry_2007']
###### Cluster 2
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명: ['bathroom_bestwestern_hotel_sfo', 'food_swissotel_chicago', 'food_holiday_inn_london', 'free_bestwestern_hotel_sfo', '